In [ ]:
import numpy as np
import pandas as pd
import os
import joblib  
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.regularizers import l1_l2

from sklearn.metrics import classification_report

# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:

# 1. Cargar el modelo entrenado
model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/metadata_model.pkl'  # Reemplaza con la ruta de tu modelo
modelo = joblib.load(model_path)  # Cargar el modelo entrenado

# 2. Cargar el dataset de entrada (con los metadatos que usarás para predecir)
dataset_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/dataset_flask.csv'  # Reemplaza con la ruta del dataset
df = pd.read_csv(dataset_path)

# Asegúrate de que 'X' sea el conjunto de características sobre las cuales el modelo predice
X = df.drop(columns=['isic_id', 'target'])  # Eliminar columnas no relevantes para la predicción

# 3. Hacer predicciones crudas (probabilidades)
predicciones_crudas = modelo.predict_proba(X)  # Obtenemos las probabilidades

# Si es un problema binario, puedes querer la probabilidad solo de la clase positiva
# (asumiendo que la clase positiva es la segunda columna de la matriz de probabilidades)
probabilidades_clase_positiva = predicciones_crudas[:, 1]

# 4. Crear un nuevo dataframe con 'isic_id', 'target' y las predicciones crudas
df_resultados = df[['isic_id', 'target']].copy()  # Copiar columnas 'isic_id' y 'target'
df_resultados['prediccion_cruda'] = probabilidades_clase_positiva  # Añadir columna con las probabilidades crudas

# 5. Guardar el nuevo dataset con las predicciones crudas
output_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones.csv'  # Define dónde guardar el nuevo archivo
df_resultados.to_csv(output_path, index=False)

print(f"El archivo con las predicciones crudas ha sido guardado en {output_path}")

El archivo con las predicciones crudas ha sido guardado en /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones.csv


Haremos una nueva prueba de entrenamiento del modelo de imagenes resnet50 añadiendo la nueva columna como caracteristica

In [17]:
# Ruta donde se encuentran las imágenes y el archivo con predicciones crudas
image_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/test web/jpg_ensambled_model'

# Cargar los metadatos
metadata_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/data/nuevo_dataset_con_predicciones.csv'
metadata = pd.read_csv(metadata_path)


In [18]:

# Listar los dispositivos físicos disponibles, en este caso, GPUs
physical_devices = tf.config.list_physical_devices('GPU')

# Si hay alguna GPU disponible
if len(physical_devices) > 0:
    # Permitir que TensorFlow crezca dinámicamente la memoria utilizada en la GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [19]:
# Convertir la columna 'target' a string
metadata['target'] = metadata['target'].astype(str)

# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(metadata, test_size=0.2, stratify=metadata['target'], random_state=42)

print(train_df.shape)
print(val_df.shape)

(711, 3)
(178, 3)


In [20]:


# Generador personalizado para combinar imágenes y predicción cruda
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, image_dir, batch_size, target_size, shuffle=True, mode='train'):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.mode = mode  # Puede ser 'train' o 'val'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_data = self.dataframe.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        images = np.array([self._load_image(file_name) for file_name in batch_data['isic_id']])
        predicciones_crudas = np.array(batch_data['prediccion_cruda']).reshape(-1, 1)
        labels = np.array(batch_data['target']).astype('float32').reshape(-1, 1)

        if self.mode == 'train':
            return [images, predicciones_crudas], labels
        else:
            return [images, predicciones_crudas]

    def on_epoch_end(self):
        if self.shuffle:
            self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)

    def _load_image(self, image_name):
        img_path = os.path.join(self.image_dir, image_name + '.jpg')
        img = load_img(img_path, target_size=self.target_size)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  # Aplicar normalización específica de ResNet50
        return img_array

# Crear los generadores de entrenamiento y validación
train_gen = CustomDataGenerator(train_df, image_dir=image_path, batch_size=32, target_size=(256, 256), shuffle=True)
val_gen = CustomDataGenerator(val_df, image_dir=image_path, batch_size=32, target_size=(256, 256), shuffle=False)

In [21]:

# Cargar el modelo preentrenado ResNet50 (sin la parte superior)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Entrada de la imagen
image_input = layers.Input(shape=(256, 256, 3), name='image_input')
x = base_model(image_input)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Entrada de la predicción cruda
prediccion_cruda_input = layers.Input(shape=(1,), name='prediccion_cruda_input')

# Concatenar ambas entradas
combined = layers.Concatenate()([x, prediccion_cruda_input])

# Capa de salida
output = layers.Dense(1, activation='sigmoid', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(combined)

# Definir el modelo
model = models.Model(inputs=[image_input, prediccion_cruda_input], outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Resumen del modelo
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 8, 8, 2048)           2358771   ['image_input[0][0]']         
                                                          2                                       
                                                                                                  
 flatten_2 (Flatten)         (None, 131072)               0         ['resnet50[0][0]']            
                                                                                                  
 dense_4 (Dense)             (None, 128)                  1677734   ['flatten_2[0][0]']     

In [22]:
# Entrenamiento inicial (solo las capas superiores)
history = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)

# Descongelar algunas capas del modelo base para fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compilar de nuevo con una tasa de aprendizaje más baja
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Calcular los pesos de clase
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_df['target']), y=train_df['target'])
class_weights = dict(enumerate(class_weights))

# Entrenamiento con Fine-Tuning
history_fine = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

Epoch 1/5


2024-09-06 19:19:25.108762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - ETA: 0s - loss: 0.9405 - accuracy: 0.6222 - precision_3: 0.5714 - recall_3: 0.5677 - auc: 0.6572

2024-09-06 19:19:30.795350: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 9s 296ms/step - loss: 0.9405 - accuracy: 0.6222 - precision_3: 0.5714 - recall_3: 0.5677 - auc: 0.6572 - val_loss: 0.5140 - val_accuracy: 0.7500 - val_precision_3: 0.6742 - val_recall_3: 0.8451 - val_auc: 0.8588
Epoch 2/5
22/22 [==============================] - 5s 239ms/step - loss: 0.7209 - accuracy: 0.7088 - precision_3: 0.6770 - recall_3: 0.6396 - auc: 0.7692 - val_loss: 0.4828 - val_accuracy: 0.8000 - val_precision_3: 0.7241 - val_recall_3: 0.8873 - val_auc: 0.8891
Epoch 3/5
22/22 [==============================] - 5s 239ms/step - loss: 0.5075 - accuracy: 0.7628 - precision_3: 0.7296 - recall_3: 0.7273 - auc: 0.8520 - val_loss: 0.4250 - val_accuracy: 0.8188 - val_precision_3: 0.7625 - val_recall_3: 0.8592 - val_auc: 0.8977
Epoch 4/5
22/22 [==============================] - 5s 240ms/step - loss: 0.5361 - accuracy: 0.7727 - precision_3: 0.7500 - recall_3: 0.7258 - auc: 0.8459 - val_loss: 0.3907 - val_accuracy: 0.8000 - val_precision_3: 0.8000

Epoch 1/10


2024-09-06 19:19:55.188597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - ETA: 0s - loss: 0.7388 - accuracy: 0.7358 - precision_4: 0.7003 - recall_4: 0.6958 - auc: 0.8059

2024-09-06 19:20:01.940491: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 10s 343ms/step - loss: 0.7388 - accuracy: 0.7358 - precision_4: 0.7003 - recall_4: 0.6958 - auc: 0.8059 - val_loss: 0.4068 - val_accuracy: 0.8375 - val_precision_4: 0.8358 - val_recall_4: 0.7887 - val_auc: 0.9085
Epoch 2/10
22/22 [==============================] - 6s 274ms/step - loss: 0.5074 - accuracy: 0.8082 - precision_4: 0.7599 - recall_4: 0.8170 - auc: 0.8799 - val_loss: 0.4083 - val_accuracy: 0.8250 - val_precision_4: 0.7867 - val_recall_4: 0.8310 - val_auc: 0.9069
Epoch 3/10
22/22 [==============================] - 6s 271ms/step - loss: 0.3532 - accuracy: 0.8622 - precision_4: 0.8397 - recall_4: 0.8479 - auc: 0.9265 - val_loss: 0.4253 - val_accuracy: 0.8500 - val_precision_4: 0.8133 - val_recall_4: 0.8592 - val_auc: 0.9088
Epoch 4/10
22/22 [==============================] - 6s 278ms/step - loss: 0.3339 - accuracy: 0.8665 - precision_4: 0.8267 - recall_4: 0.8803 - auc: 0.9370 - val_loss: 0.4916 - val_accuracy: 0.8250 - val_precision_4: 0.

In [26]:

# Función para recoger todas las etiquetas verdaderas del generador
def get_all_labels(generator):
    labels = []
    for i in range(len(generator)):
        _, batch_labels = generator[i]
        labels.extend(batch_labels)
    return np.array(labels).flatten()

# Evaluar el modelo
loss, accuracy, precision, recall, auc = model.evaluate(val_gen)
print(f"Loss: {loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}")

# Predecir las probabilidades
y_pred_probs = model.predict(val_gen)

# Ajustar el umbral de decisión
threshold = 0.5
y_pred_adjusted = (y_pred_probs > threshold).astype(int).flatten()

# Obtener las etiquetas verdaderas desde el generador
y_true = get_all_labels(val_gen)

# Asegurarse de que el número de predicciones coincida con el número de etiquetas verdaderas
assert len(y_pred_adjusted) == len(y_true), "El número de predicciones no coincide con el número de etiquetas reales."

# Calcular y mostrar el reporte de clasificación
print(classification_report(y_true, y_pred_adjusted, target_names=['Class 0', 'Class 1']))

5/5 [==============================] - 1s 189ms/step - loss: 0.4068 - accuracy: 0.8375 - precision_4: 0.8358 - recall_4: 0.7887 - auc: 0.9085
Loss: 0.4068334102630615, Accuracy: 0.8374999761581421, Precision: 0.8358209133148193, Recall: 0.7887324094772339, AUC: 0.9084506630897522
5/5 [==============================] - 1s 176ms/step
              precision    recall  f1-score   support

     Class 0       0.84      0.88      0.86        89
     Class 1       0.84      0.79      0.81        71

    accuracy                           0.84       160
   macro avg       0.84      0.83      0.83       160
weighted avg       0.84      0.84      0.84       160



salida del entrenamiento del modelo de imagenes con el añadido de la columna de resultado del modleo entrenado solo con mtadatos

5/5 [==============================] - 1s 189ms/step - loss: 0.4068 - accuracy: 0.8375 - precision_4: 0.8358 - recall_4: 0.7887 - auc: 0.9085
Loss: 0.4068334102630615, Accuracy: 0.8374999761581421, Precision: 0.8358209133148193, Recall: 0.7887324094772339, AUC: 0.9084506630897522
5/5 [==============================] - 1s 176ms/step
              precision    recall  f1-score   support

     Class 0       0.84      0.88      0.86        89
     Class 1       0.84      0.79      0.81        71

    accuracy                           0.84       160
   macro avg       0.84      0.83      0.83       160
weighted avg       0.84      0.84      0.84       160



In [27]:
# Ruta donde guardar el modelo
model_save_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl'

# Guardar el modelo
model.save(model_save_path)
print(f"Modelo guardado en {model_save_path}")

INFO:tensorflow:Assets written to: /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl/assets


INFO:tensorflow:Assets written to: /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl/assets


Modelo guardado en /Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer_jpg_data/static/model/prueba_modelo_encadenado.pkl
